In [1]:
import numpy as np
import os
import json
import numpy as np
import random
import torch
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
from scipy.signal import sawtooth, butter, lfilter
from IPython.display import Audio
from tqdm import tqdm

In [2]:
import homework5

In [3]:
dataroot = "/home/julian/Downloads"

### Load JSON files that contain information about existing playlists

In [4]:
train_playlists = json.load(open(dataroot + '/' + "train_playlists.json"))
train_playlists['0']

[{'tid': 'TRCSBUG128F422318C',
  'artist_name': 'Billy Talent',
  'track_name': 'The Navy Song'},
 {'tid': 'TRRJQTI128F4223186',
  'artist_name': 'Billy Talent',
  'track_name': 'Devil In A Midnight Mass'},
 {'tid': 'TRCHBZS128F92C44C6',
  'artist_name': 'Scary Kids Scaring Kids',
  'track_name': 'A Pistol To My Temple'},
 {'tid': 'TRMKOMO128F14A5A8C',
  'artist_name': 'Lovehatehero',
  'track_name': 'Goodbye My Love'},
 {'tid': 'TRDQOOL12903CF81CD',
  'artist_name': 'Dance Gavin Dance',
  'track_name': 'Uneasy Hearts Weigh The Most'}]

In [5]:
test_playlists = json.load(open(dataroot + '/' + "test_playlists.json"))
test_playlists['0']

[{'tid': 'TRPNPTL128EF35F8B6',
  'artist_name': 'Maroon 5',
  'track_name': 'Makes Me Wonder'},
 {'tid': 'TROUVLE128F9345091',
  'artist_name': 'Jeremih',
  'track_name': 'Birthday Sex'},
 {'tid': 'TRYPSFO12903CA84F0',
  'artist_name': 'The Pussycat Dolls',
  'track_name': 'Stickwitu'},
 {'tid': 'TRRORBU128F93113D1',
  'artist_name': 'Rihanna',
  'track_name': 'Rehab'},
 {'tid': 'TRSSPPR128F92C3A4E',
  'artist_name': 'The Pussycat Dolls',
  'track_name': 'When I Grow Up'},
 {'tid': 'TRDLKVL128F1483283',
  'artist_name': 'Fergie',
  'track_name': 'Clumsy'},
 {'tid': 'TRKBWUX128F92D54F3',
  'artist_name': 'Jordin Sparks',
  'track_name': 'Tattoo'},
 {'tid': 'TRONBHE128F92D6309',
  'artist_name': 'Katy Perry',
  'track_name': 'I Kissed A Girl'},
 {'tid': 'TRCNVLA12903CF6052',
  'artist_name': 'Kelis',
  'track_name': 'Milkshake'},
 {'tid': 'TRGDVRO128F92F5994',
  'artist_name': 'The Pussycat Dolls',
  'track_name': 'I Hate This Part'}]

In [6]:
data, tid_to_idx, idx_to_tid, tid_to_meta = homework5.build_interaction_samples(train_playlists)
len(data)

297674

In [7]:
BATCH_SIZE = 1024
dataset = homework5.WRMFDataset(data)
dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)

In [8]:
model = homework5.WRMF(len(train_playlists), len(tid_to_idx), num_factors=16)
optimizer = optim.Adam(model.parameters(), lr=0.01)

In [9]:
model.train()

for epoch in tqdm(range(10)):  
    total_loss = 0
    for user, item, label in dataloader:
        optimizer.zero_grad()
        loss = model.compute_loss(user, item, label)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
        
    print(f"Epoch {epoch}: Loss = {total_loss/len(dataloader):.4f}")

 10%|████▍                                       | 1/10 [00:01<00:17,  1.91s/it]

Epoch 0: Loss = 4.7994


 20%|████████▊                                   | 2/10 [00:03<00:15,  1.88s/it]

Epoch 1: Loss = 2.1240


 30%|█████████████▏                              | 3/10 [00:05<00:12,  1.84s/it]

...

Epoch 7: Loss = 1.0988


 90%|███████████████████████████████████████▌    | 9/10 [00:16<00:01,  1.86s/it]

Epoch 8: Loss = 1.0953


100%|███████████████████████████████████████████| 10/10 [00:18<00:00,  1.86s/it]

Epoch 9: Loss = 1.0928


In [10]:
# Note: loss should be ~1 after 10 epochs

In [11]:
playlist1 = ['TRUELCH128F92C4797', 'TRXPADO128F42A964A'] #Britney Spears, Kelly Clarkson
playlist2 = ['TRZSOWU128F9310EA2', 'TRTAFRC128F932B6F9'] #Radiohead, Muse
playlist3 = ['TRDTGYF128F4214DAA', 'TRJNNBG12903CD3AEE'] #Dr. Dre, Eric B. & Rakim
playlist4 = ['TRKCZYQ128F426B019', 'TRBLANS128F92FD3FF'] #Rage Against The Machine,Audioslave

In [12]:
model.eval()
all_item_embeddings = model.item_factors.weight

In [13]:
sc, recs = homework5.generate_recommendations(model, playlist1, all_item_embeddings, idx_to_tid, tid_to_idx, 10)

In [14]:
sc, recs

([0.9996631145477295,
  0.9996379613876343,
  0.9996293783187866,
  0.9996281862258911,
  0.9994980692863464,
  0.9994973540306091,
  0.999127984046936,
  0.9990848898887634,
  0.9987831711769104,
  0.9986680746078491],
 ['TRWDJRQ128F4295E13',
  'TRETMNQ128F9323A27',
  'TRSIYFY128F932FE88',
  'TRJBSYG128F4291E11',
  'TRQMELD128F930BE43',
  'TRBAUVN128F932FEF8',
  'TRMEBYQ128F932FEFF',
  'TRONBHE128F92D6309',
  'TRUZAOX128F9305498',
  'TROIZTP128F9304382'])

In [15]:
[tid_to_meta[x] for x in recs]

[('Kelly Clarkson', 'Miss Independent'),
 ('Jordin Sparks', 'Battlefield'),
 ('Britney Spears', 'Stronger'),
 ('Britney Spears', 'Lucky'),
 ('Avril Lavigne', 'Sk8er Boi'),
 ('Britney Spears', 'Oops!...I Did It Again'),
 ('Britney Spears', 'Toxic'),
 ('Katy Perry', 'I Kissed A Girl'),
 ('Justin Timberlake', 'Rock Your Body'),
 ('Kelly Clarkson', 'I Do Not Hook Up')]

In [16]:
cf_rankings, cf_targets = homework5.make_cf_rankings(model, test_playlists, all_item_embeddings, idx_to_tid, tid_to_idx)

In [17]:
cf_targets['0']

['TRYPSFO12903CA84F0',
 'TRRORBU128F93113D1',
 'TRSSPPR128F92C3A4E',
 'TRDLKVL128F1483283',
 'TRKBWUX128F92D54F3',
 'TRONBHE128F92D6309',
 'TRCNVLA12903CF6052',
 'TRGDVRO128F92F5994']

In [18]:
cf_mrr = homework5.get_mrr(cf_rankings, cf_targets)
cf_precision = homework5.get_precision(cf_rankings, cf_targets)
cf_mrr, cf_precision

(np.float64(0.08271794998171247), np.float64(0.1494563492063492))

In [20]:
playlist_embedding = homework5.get_average_audio_embedding(playlist1, dataroot + '/' + "audio_embeddings/")
tids = list(tid_to_meta.keys())
embedding_matrix, tids = homework5.get_embeddings(dataroot + '/' + "audio_embeddings/", tids)

In [21]:
playlist1

['TRUELCH128F92C4797', 'TRXPADO128F42A964A']

In [22]:
sims, ranking = homework5.get_similarity(playlist1, playlist_embedding, embedding_matrix, tid_to_idx, tids)

In [23]:
sims, ranking

([0.9576328992843628,
  0.9526731967926025,
  0.936994194984436,
  0.9366549849510193,
  0.9364639520645142,
  0.9291771650314331,
  0.927935779094696,
  0.9267312288284302,
  0.9263182878494263,
  0.9210880994796753,
  0.9208636283874512,
  0.9181893467903137,
  0.9154109954833984,
  0.9151930809020996,
  0.9131646156311035,
  0.906150221824646,
  0.9060567617416382,
  0.9044654965400696,
  0.9017198085784912,
  0.9009044170379639,
  0.8999133706092834,
  0.8975818753242493,
  0.8973529934883118,
  0.8971996307373047,
  0.8969649076461792,
  0.8906939029693604,
  0.8890079259872437,
  0.8887665271759033,
  0.8870862126350403,
  0.8865243792533875,
  0.8862819671630859,
  0.885858416557312,
  0.8853836059570312,
  0.8837671279907227,
  0.8829200267791748,
  0.8828030824661255,
  0.8824011087417603,
  0.8816651701927185,
  0.8807753920555115,
  0.8806085586547852,
  0.8797284364700317,
  0.8796195983886719,
  0.8793390989303589,
  0.8784074187278748,
  0.8782084584236145,
  0.8781373500

In [25]:
audio_rankings, audio_targets =\
  homework5.make_audio_rankings(dataroot + '/' + "audio_embeddings/",\
                                test_playlists, embedding_matrix, tid_to_idx, tids)

In [26]:
homework5.get_similarity(playlist1, playlist_embedding, embedding_matrix, tid_to_idx, tids)

([0.9576328992843628,
  0.9526731967926025,
  0.936994194984436,
  0.9366549849510193,
  0.9364639520645142,
  0.9291771650314331,
  0.927935779094696,
  0.9267312288284302,
  0.9263182878494263,
  0.9210880994796753,
  0.9208636283874512,
  0.9181893467903137,
  0.9154109954833984,
  0.9151930809020996,
  0.9131646156311035,
  0.906150221824646,
  0.9060567617416382,
  0.9044654965400696,
  0.9017198085784912,
  0.9009044170379639,
  0.8999133706092834,
  0.8975818753242493,
  0.8973529934883118,
  0.8971996307373047,
  0.8969649076461792,
  0.8906939029693604,
  0.8890079259872437,
  0.8887665271759033,
  0.8870862126350403,
  0.8865243792533875,
  0.8862819671630859,
  0.885858416557312,
  0.8853836059570312,
  0.8837671279907227,
  0.8829200267791748,
  0.8828030824661255,
  0.8824011087417603,
  0.8816651701927185,
  0.8807753920555115,
  0.8806085586547852,
  0.8797284364700317,
  0.8796195983886719,
  0.8793390989303589,
  0.8784074187278748,
  0.8782084584236145,
  0.8781373500

In [27]:
audio_mrr = homework5.get_mrr(audio_rankings, audio_targets)
audio_precision = homework5.get_precision(audio_rankings, audio_targets)
audio_mrr, audio_precision

(np.float64(0.010380286135487067), np.float64(0.017666666666666667))

In [28]:
DURATION = 5
SAMPLE_RATE = 44100
ATTACK_TIME = 0.5
DECAY_TIME = 0.5
SUSTAIN_LEVEL = 0.2
RELEASE_TIME = 1.0
t = np.linspace(0, DURATION + RELEASE_TIME, int(SAMPLE_RATE * (DURATION + RELEASE_TIME)), endpoint=False)
wave = sawtooth(2 * np.pi * 440 * t)

env = homework5.adsr_envelope(DURATION, ATTACK_TIME, DECAY_TIME, SUSTAIN_LEVEL, RELEASE_TIME, SAMPLE_RATE)

Audio(wave * env, rate=SAMPLE_RATE)

In [29]:
SAMPLE_RATE = 44100
DURATION = 3  
BASE_CUTOFF = 1000 
LFO_FREQ = 0.5
LFO_DEPTH = 440  

t = np.linspace(0, DURATION, int(SAMPLE_RATE * DURATION), endpoint=False)

lfo = homework5.get_lfo(BASE_CUTOFF, LFO_DEPTH, LFO_FREQ, t)

saw = sawtooth(2 * np.pi * 220 * t)
frame_size = 1024
filtered = np.zeros_like(saw)

for i in range(0, len(saw), frame_size):
    frame = saw[i:i + frame_size]
    cutoff = lfo[i]
    if len(frame) < frame_size:
        break 
    norm_cutoff = cutoff / (SAMPLE_RATE / 2)
    b, a = butter(N=2, Wn=norm_cutoff, btype='low', analog=False)
    filtered_frame = lfilter(b, a, frame)
    
    filtered[i:i + frame_size] = filtered_frame

Audio(filtered, rate=SAMPLE_RATE)